In [1]:
!git clone https://github.com/Chinnu1103/Machine-Translation-using-Transformers.git

Cloning into 'Machine-Translation-using-Transformers'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 34 (delta 1), reused 5 (delta 1), pack-reused 26
Unpacking objects: 100% (34/34), done.
Checking out files: 100% (11/11), done.


In [0]:
!cp -r /content/Machine-Translation-using-Transformers/* .

###Import all the necessary modules

In [0]:
from os.path import isfile
import tensorflow as tf
import numpy as np
import pickle
import time
import re

###Choose Translate direction

In [0]:
#Set htoe as true for translation from Hindi to Hnglish; 
#Set htoe as false for English to Hindi translation
htoe = True

###Load the Tokenizers

In [0]:
if htoe:
    tokenizer_one = pickle.load(open("tokenizer_hi","rb"))
    tokenizer_two = pickle.load(open("tokenizer_en","rb"))
else:
    tokenizer_one = pickle.load(open("tokenizer_en","rb"))
    tokenizer_two = pickle.load(open("tokenizer_hi","rb"))

###Create the dataset - **For training purposes only**

In [0]:
# Tweak the dataset params according to your needs

MAX_LENGTH = 64 
BUFFER_SIZE = 1000
BATCH_SIZE = 100

In [32]:
# Read the dataset objects

if htoe:
    raw_data_one = pickle.load(open("raw_data_hi","rb"))
    raw_data_two = pickle.load(open("raw_data_en","rb"))
else:
    raw_data_one = pickle.load(open("raw_data_en","rb"))
    raw_data_two = pickle.load(open("raw_data_hi","rb"))

print("Dataset size: {}".format(len(raw_data_one)))

Dataset size: 690722


In [0]:
# Utility functions for data preprocessing

def encode(lang1, lang2):
  lang1 = [len(tokenizer_one.word_index)] + tokenizer_one.texts_to_sequences(
      [lang1.numpy().decode("utf-8")])[0] + [len(tokenizer_one.word_index)+1]

  lang2 = [len(tokenizer_two.word_index)] + tokenizer_two.texts_to_sequences(
      [lang2.numpy().decode("utf-8")])[0] + [len(tokenizer_two.word_index)+1]
  
  return lang1, lang2

def tf_encode(lang1, lang2):
  result_one, result_two = tf.py_function(
      encode, [lang1, lang2], [tf.int64, tf.int64])
  result_one.set_shape([None])
  result_two.set_shape([None])

  return result_one, result_two

def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [0]:
# Obtain tf-data object from the dataset

train_examples = tf.data.Dataset.from_tensor_slices((raw_data_one, raw_data_two)) 

train_preprocessed = (
    train_examples 
    .map(tf_encode)
    .filter(filter_max_length)
    .cache()
    .shuffle(BUFFER_SIZE))

train_dataset = (train_preprocessed
                 .padded_batch(BATCH_SIZE)
                 .prefetch(tf.data.experimental.AUTOTUNE))

###Create the word embedding matrices

In [0]:
# Define the vocab sizes

input_vocab_size = len(tokenizer_one.word_index) + 2
target_vocab_size = len(tokenizer_two.word_index) + 2

In [0]:
# Load the pretrained embeddings

words_en, embeddings_en = pickle.load(
    open('polyglot-en.pkl', 'rb'), encoding='latin1')
words_hi, embeddings_hi = pickle.load(
    open('polyglot-hi.pkl', 'rb'), encoding='latin1')

In [0]:
# English embedding matrix

embeddings_index_en = {}
if htoe:
    word_index_en = tokenizer_two.word_index
else:
    word_index_en = tokenizer_one.word_index

for i in range(len(words_en)):
    embeddings_index_en[words_en[i].lower()] = embeddings_en[i]

if htoe:
    embedding_matrix_en = np.zeros((target_vocab_size, 64))
else:
    embedding_matrix_en = np.zeros((input_vocab_size, 64))

for word, i in word_index_en.items():
    embedding_vector = embeddings_index_en.get(word)
    if embedding_vector is not None:
        embedding_matrix_en[i] = embedding_vector

In [0]:
# Hindi embedding matrix

embeddings_index_hi = {}
if htoe:
    word_index_hi = tokenizer_one.word_index
else:
    word_index_hi = tokenizer_two.word_index

for i in range(len(words_hi)):
    embeddings_index_hi[words_hi[i]] = embeddings_hi[i]

if htoe:
    embedding_matrix_hi = np.zeros((input_vocab_size, 64))
else:
    embedding_matrix_hi = np.zeros((target_vocab_size, 64))
    
for word, i in word_index_hi.items():
    embedding_vector = embeddings_index_hi.get(word)
    if embedding_vector is not None:
        embedding_matrix_hi[i] = embedding_vector

In [0]:
if htoe:
    embedding_matrix_one = embedding_matrix_hi
    embedding_matrix_two = embedding_matrix_en
else:
    embedding_matrix_one = embedding_matrix_en
    embedding_matrix_two = embedding_matrix_hi

###Create the model

In [0]:
# Set the model Hyperparams

num_layers = 6
d_model = 64
dff = 2048
num_heads = 32
dropout_rate = 0.1

In [0]:
# Utility functions for the model

def positional_encoding(max_pos, d_model):
    """ Returns the positional encoding for all positions

    Args:
        max_pos: (int) size of required positional embeddings equal to 
        the vocab size
        d_model: (int) model size equal to the embedding size
    
    Returns:
        pe: (tensor of type float32, shape = 
        (1, max_pos, d_model)) positional encodings of type float32
    """

    theta = np.expand_dims(np.arange(max_pos), 1) / (np.power(10000, 
    (2 * np.expand_dims(np.arange(d_model), 0) // 2) / d_model))
  
    # sin(i) for all even i
    theta[:, 0::2] = np.sin(theta[:, 0::2]) 
    
    # cos(i) for all odd i
    theta[:, 1::2] = np.cos(theta[:, 1::2]) 
    
    pe = np.expand_dims(theta, 0)
    
    return tf.cast(pe, dtype=tf.float32)

def create_padding_mask(seq):
    """ Creates a padding mask for the self attention layer in the decoder
    """

    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    
    # (batch_size, 1, 1, seq_len)
    return seq[:, tf.newaxis, tf.newaxis, :]  

def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  
  # (seq_len, seq_len)
  return mask  

def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  # (..., seq_len_q, seq_len_k)
  matmul_qk = tf.matmul(q, k, transpose_b=True)  
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  # (..., seq_len_q, seq_len_k)
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  

  # (..., seq_len_q, depth_v)
  output = tf.matmul(attention_weights, v)  

  return output, attention_weights

def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('Attention weights are:')
  print (temp_attn)
  print ('Output is:')
  print (temp_out)

def point_wise_feed_forward_network(d_model, dff):

    # (batch_size, seq_len, dff)
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)
  
  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)
  
  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by 
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

In [0]:
# Define the Model layers

# Multihead Attention keras layer
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is :
    (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    # (batch_size, seq_len, d_model)
    q = self.wq(q)  
    k = self.wk(k)
    v = self.wv(v)
    
    # (batch_size, num_heads, seq_len_q, depth)
    q = self.split_heads(q, batch_size)

    # (batch_size, num_heads, seq_len_k, depth)  
    k = self.split_heads(k, batch_size)

    # (batch_size, num_heads, seq_len_v, depth)
    v = self.split_heads(v, batch_size)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    # (batch_size, seq_len_q, num_heads, depth)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

    # (batch_size, seq_len_q, d_model)
    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

# Encoder keras Layer

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    # (batch_size, input_seq_len, d_model)
    attn_output, _ = self.mha(x, x, x, mask)  
    attn_output = self.dropout1(attn_output, training=training)
    
    # (batch_size, input_seq_len, d_model)
    out1 = self.layernorm1(x + attn_output)  
    
    # (batch_size, input_seq_len, d_model)
    ffn_output = self.ffn(out1)  
    ffn_output = self.dropout2(ffn_output, training=training)
    
    # (batch_size, input_seq_len, d_model)
    out2 = self.layernorm2(out1 + ffn_output)  
    
    return out2

class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(
        input_vocab_size, 
        d_model, 
        weights = [embedding_matrix_one], 
        trainable = False)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    # (batch_size, input_seq_len, d_model)
    x = self.embedding(x)  
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    # (batch_size, input_seq_len, d_model)
    return x  

# Decoder keras layer

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    # (batch_size, target_seq_len, d_model)
    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)
    
    # (batch_size, target_seq_len, d_model)
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  
    attn2 = self.dropout2(attn2, training=training)
    
    # (batch_size, target_seq_len, d_model)
    out2 = self.layernorm2(attn2 + out1)  
    
    # (batch_size, target_seq_len, d_model)
    ffn_output = self.ffn(out2)  
    ffn_output = self.dropout3(ffn_output, training=training)
    
    # (batch_size, target_seq_len, d_model)
    out3 = self.layernorm3(ffn_output + out2)  
    
    return out3, attn_weights_block1, attn_weights_block2

class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(
        target_vocab_size, 
        d_model, 
        weights = [embedding_matrix_two], 
        trainable=False)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    # (batch_size, target_seq_len, d_model)
    x = self.embedding(x) 
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [0]:
# Define the Model

class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

    # (batch_size, inp_seq_len, d_model)
    enc_output = self.encoder(inp, training, enc_padding_mask)  
    
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    # (batch_size, tar_seq_len, target_vocab_size)
    final_output = self.final_layer(dec_output)  
    
    return final_output, attention_weights

In [0]:
# Define the Learning Rate

# Custom Learning rate scheduler

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# create the optimizer object with the custom learning rate

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, 
    beta_1=0.9, 
    beta_2=0.98, 
    epsilon=1e-9)

In [0]:
# Define the loss for the model

# Create the required type of loss

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, 
    reduction='none')

# Define how loss is calculated

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

# Create the loss and accuracy objects

train_loss = tf.keras.metrics.Mean(
    name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [46]:
# Create the model object with the necessary params

# Unzip the weights

!unzip -o transformer_htoe.weights.zip
!unzip -o transformer_etoh.weights.zip

transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

# Load any pre-trained weights (Comment out if retraining from scratch)

if htoe:
    transformer.load_weights('transformer_htoe.weights')
else:
    transformer.load_weights('transformer_etoh.weights')

Archive:  transformer_htoe.weights.zip
  inflating: transformer_htoe.weights.data-00000-of-00002  
  inflating: transformer_htoe.weights.data-00001-of-00002  
  inflating: transformer_htoe.weights.index  
Archive:  transformer_etoh.weights.zip
  inflating: transformer_etoh.weights.data-00000-of-00002  
  inflating: transformer_etoh.weights.data-00001-of-00002  
  inflating: transformer_etoh.weights.index  


###Train the model - Not needed for predictions only

In [0]:
# Define training epochs

EPOCHS = 2

In [0]:
# Define the training step function

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

In [49]:
# Start training the model

for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  for (batch, (inp, tar)) in enumerate(train_dataset):
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
    
  transformer.save_weights('transformer_{}.weights'.format(epoch + 1))
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.6018 Accuracy 0.1076


KeyboardInterrupt: ignored

###Predict sentences using the model

In [0]:
# Utility functions for prediction

MAX_LENGTH = 64 

def preprocess_string(s):
    ''' String preprocessing function

    Args:
        s: The string to be preprocessed
    
    Returns:
        s: The preprocessed String
    '''

    if htoe:
        s = re.sub(r'[a-zA-Z]', '', s) # Removes english chars from hindi text
    s = re.sub(r"[\(\[].*?[\)\]]", "", s) # Removes text between braces
    s = re.sub(r'([!.?।])', r' \1', s) #Includes space between some characters
    s = re.sub(r'\s+', r' ', s) #Reduces multispace string to a single space
    
    return s

def evaluate(inp_sentence):
  start_token = [len(tokenizer_one.word_index)]
  end_token = [len(tokenizer_one.word_index) + 1]
  
  inp_sentence = start_token + tokenizer_one.texts_to_sequences(
      [inp_sentence])[0] + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)
  
  decoder_input = [len(tokenizer_two.word_index)]
  output = tf.expand_dims(decoder_input, 0)
    
  for i in range(64):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # return the result if the predicted_id is equal to the end token
    if predicted_id == len(tokenizer_two.word_index)+1:
      return tf.squeeze(output, axis=0), attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0), attention_weights

def translate(sentence):
  result, attention_weights = evaluate(sentence)
  predicted_sentence = tokenizer_two.sequences_to_texts(
      [[i.numpy()] for i in result if i < len(tokenizer_two.word_index)])  

  return ' '.join(predicted_sentence)

In [52]:
#Enter input sentence
inp_str = "क्या मैं आपकी मदद कर सकता हुँ ?"

print(translate(preprocess_string(inp_str)))

can i help you ?
